<a href="https://colab.research.google.com/github/Nattakonjpg/Project-Valolyze/blob/main/Predict_MoViNets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Google Drive

In [1]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive
%ls

/content/drive/MyDrive
 A5_yoru+habor_Checkpoint_epoch_round112.pth  'Csv file'/     transforms.py
'Colab Notebooks'/                             __pycache__/  'video แบ่งรอบไม่ตัด'/


## Install package Library

###Install Package

In [ ]:
!pip install av

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 21.3 MB/s eta 0:00:00


In [ ]:
pip install git+https://github.com/Atze00/MoViNet-pytorch.git

  Cloning https://github.com/Atze00/MoViNet-pytorch.git to /tmp/pip-req-build-x27a8ay2
  Running command git clone --filter=blob:none --quiet https://github.com/Atze00/MoViNet-pytorch.git /tmp/pip-req-build-x27a8ay2
  Resolved https://github.com/Atze00/MoViNet-pytorch.git to commit c2d1edf48fc6c5259707f9d833f22171b4f63493
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for MoViNet-pytorch: filename=MoViNet_pytorch-0.2.2-py3-none-any.whl size=11959 sha256=91aeb58716554ad51f3a586216fd35855e7e05fd85dd7742a4f35e278a3e2fb9
  Stored in directory: /tmp/pip-ephem-wheel-cache-n3oe67h3/wheels/a4/76/cf/b8a087460d770e5a44eff9ed975e225d43f007c8c14f8e26d2
  Created wheel for

In [ ]:
import time
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import random_split, DataLoader
import torch
import transforms as T
from movinets import MoViNet
from movinets.config import _C

#Model

##Install ffmpeg


In [ ]:
from IPython.display import clear_output
!sudo curl -L https://github.com/BtbN/FFmpeg-Builds/releases/download/latest/ffmpeg-master-latest-linux64-gpl.tar.xz -o /usr/local/bin/ffmpeg.tar.xz
clear_output()
%cd /usr/local/bin/
clear_output()
!7z e /usr/local/bin/ffmpeg.tar.xz
clear_output()
!7z e /usr/local/bin/ffmpeg.tar
clear_output()
!sudo chmod a+rx /usr/local/bin/ffmpeg
clear_output()
%cd /content/
!sudo curl -L https://mkvtoolnix.download/appimage/MKVToolNix_GUI-70.0.0-x86_64.AppImage -o /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage
!sudo chmod u+rx /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage
!sudo ln -s /usr/local/bin/MKVToolNix_GUI-70.0.0-x86_64.AppImage /usr/local/bin/mkvmerge
!sudo chmod a+rx /usr/local/bin/mkvmerge
clear_output()
!ffmpeg -version

ffmpeg version N-113725-g37702e2066-20240220 Copyright (c) 2000-2024 the FFmpeg developers
built with gcc 13.2.0 (crosstool-NG 1.25.0.232_c175b21)
configuration: --prefix=/ffbuild/prefix --pkg-config-flags=--static --pkg-config=pkg-config --cross-prefix=x86_64-ffbuild-linux-gnu- --arch=x86_64 --target-os=linux --enable-gpl --enable-version3 --disable-debug --enable-iconv --enable-libxml2 --enable-zlib --enable-libfreetype --enable-libfribidi --enable-gmp --enable-openssl --enable-lzma --enable-fontconfig --enable-libharfbuzz --enable-libvorbis --enable-opencl --enable-libpulse --enable-libvmaf --enable-libxcb --enable-xlib --enable-amf --enable-libaom --enable-libaribb24 --enable-avisynth --enable-chromaprint --enable-libdav1d --enable-libdavs2 --disable-libfdk-aac --enable-ffnvcodec --enable-cuda-llvm --enable-frei0r --enable-libgme --enable-libkvazaar --enable-libaribcaption --enable-libass --enable-libbluray --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librist --ena

### **FFMPEG**
- Path Folder video ต้นฉบับ
- Path output video
- ตัดวิดีโอต้นฉบับให้เป็นวิดีโอละ 2 วินาที
- ตั้งชื่อวิดีโอ "video_test_{i+1}.avi"

In [ ]:
import os
import subprocess

# กำหนด path ของวิดีโอที่ต้องการตัด
inputFile = "/content/drive/MyDrive/video แบ่งรอบไม่ตัด/video แบ่งรอบไม่ตัด/video_data/test_video_fullvideo/nosyy - yoru - rank immortal 1 - รอบ 1.avi"

# เรียกใช้ ffmpeg เพื่อรับข้อมูลเกี่ยวกับวิดีโอและคำนวณความยาวของมัน
command = ['ffmpeg', '-i', inputFile, '-f', 'null', '-']
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
output = result.stdout.decode('utf-8')

# หาค่า duration จาก output
duration_index = output.find('Duration: ') + len('Duration: ')
duration = output[duration_index:duration_index+11]
duration = duration.split(':')
duration = int(duration[0])*3600 + int(duration[1])*60 + int(duration[2].split('.')[0])

# กำหนดความยาวของคลิปวิดีโอที่ต้องการตัด
clipDuration = 2  # คลิปวิดีโอละ 2 วินาที

# กำหนด path ที่ต้องการบันทึกไฟล์ผลลัพธ์
outputFolder = "/content/drive/MyDrive/video แบ่งรอบไม่ตัด/video แบ่งรอบไม่ตัด/video_data/test_video_cutvideo/nosyy - yoru - rank immortal 1 - รอบ 1"

# คำนวณจำนวนคลิปวิดีโอที่ต้องการตัด
numClips = duration // clipDuration

# สร้างคำสั่ง FFMPEG สำหรับการตัดแต่ละคลิปวิดีโอ
clip_start_times = []  # เก็บเวลาเริ่มต้นของแต่ละคลิปวิดีโอ
for i in range(numClips):
    # คำนวณเวลาเริ่มต้นและสิ้นสุดของคลิปวิดีโอ
    startTime = i * clipDuration
    endTime = startTime + clipDuration

    # สร้าง command line สำหรับ FFMPEG
    commandLine = ['-hide_banner', '-i', inputFile, '-map', '0', '-c', 'copy', '-ss', "{:02d}:{:02d}:{:02d}".format(startTime // 3600, (startTime // 60) % 60, startTime % 60), '-to', "{:02d}:{:02d}:{:02d}".format(endTime // 3600, (endTime // 60) % 60, endTime % 60)]

    # กำหนด path ของไฟล์ผลลัพธ์
    outputFile = os.path.join(outputFolder, f"video_test_{i+1}.avi")
    commandLine += [outputFile]

    # ใช้ FFMPEG เพื่อตัดคลิปวิดีโอ
    subprocess.run(['ffmpeg'] + commandLine)

    # เก็บเวลาเริ่มต้นของแต่ละคลิปวิดีโอ
    clip_start_times.append("{:02d}:{:02d}:{:02d}".format(startTime // 3600, (startTime // 60) % 60, startTime % 60))

# พิมพ์เวลาเริ่มต้นของแต่ละคลิปวิดีโอ
for i, start_time in enumerate(clip_start_times):
    print(f"Clip {i+1}: {start_time}")


In [ ]:
# Create a DataFrame with clip_start_times
df = pd.DataFrame({'clip_start_times': clip_start_times})

# Save the DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Csv file/clip_start_times.csv', index=False)

###FFmpeg Create text file

In [ ]:
import os

# กำหนด path ของโฟลเดอร์ที่มีวิดีโอ
input_path =  "/content/drive/MyDrive/video แบ่งรอบไม่ตัด/video แบ่งรอบไม่ตัด/video_data/test_video_cutvideo/nosyy - yoru - rank immortal 1 - รอบ 1"

# สร้างไฟล์ text
output_file_path = "/content/drive/MyDrive/video แบ่งรอบไม่ตัด/video แบ่งรอบไม่ตัด/test_train_splits/nosyy - yoru - rank immortal 1 - รอบ 1/nosyy - yoru - rank immortal 1 - รอบ 1_test_split1.txt"
with open(output_file_path, "w") as f:
    # วนลูปผ่านไฟล์ในโฟลเดอร์
    for i, filename in enumerate(os.listdir(input_path), start=1):
        # ตรวจสอบว่าไฟล์เป็นไฟล์วิดีโอ (.avi) หรือไม่
        if filename.endswith(".avi"):
            # สร้างชื่อไฟล์ใหม่
            new_filename = f"video_test_{i}.avi 2\n"
            # เขียนชื่อไฟล์ใหม่ลงในไฟล์ text
            f.write(new_filename)


##Import Model

In [ ]:
# Define the model architecture
loaded_model1 = MoViNet(_C.MODEL.MoViNetA5, causal=False, pretrained=True)
loaded_model1.classifier[3] = torch.nn.Conv3d(2048, 8, (1, 1, 1))

# Specify the path to the checkpoint file
best_checkpoint_path = '/content/drive/MyDrive/A5_yoru+habor_Checkpoint_epoch_round112.pth'

# Load the checkpoint, specifying map_location to CPU
best_checkpoint = torch.load(best_checkpoint_path, map_location=torch.device('cpu'))

# Load the model's state dictionary
loaded_model1.load_state_dict(best_checkpoint['model_state_dict'])

# Set the model to evaluation mode
loaded_model1.eval()


###Prepare data

In [ ]:
torch.manual_seed(97)
num_frames = 24 # 16
clip_steps = 12
Bs_Train = 1
Bs_Test = 1

transform_test_A5 = transforms.Compose([
                                 T.ToFloatTensorInZeroOne(),
                                 T.Resize((400, 400)),
                                 #T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
                                 T.CenterCrop((320, 320))]) #Size for model


hmdb51_test_A5 = torchvision.datasets.HMDB51('/content/drive/MyDrive/video แบ่งรอบไม่ตัด/video แบ่งรอบไม่ตัด/video_data/test_video_cutvideo/nosyy - yoru - rank immortal 1 - รอบ 1/',
                                             '/content/drive/MyDrive/video แบ่งรอบไม่ตัด/video แบ่งรอบไม่ตัด/test_train_splits/test_video_round_1/',
                                              frames_per_clip=24,frame_rate=24,step_between_clips = 24, fold=1, train=False,transform=transform_test_A5, num_workers=2)

test_loader_A5  = DataLoader(hmdb51_test_A5 , batch_size=Bs_Test, shuffle=False)

100%|██████████| 2/2 [00:02<00:00,  1.41s/it]


In [ ]:
print(hmdb51_test_A5)

Dataset HMDB51
    Number of datapoints: 30
    Root location: /content/drive/MyDrive/video แบ่งรอบไม่ตัด/video แบ่งรอบไม่ตัด/video_data/test_video_cutvideo/nosyy - yoru - rank immortal 1 - รอบ 1/


##Predict data validation

In [ ]:
device = next(loaded_model1.parameters()).device
true_labels, predicted_labels = [], []
with torch.no_grad():
    for data, _, target in test_loader_A5:
        # Move data and target to the same device as the model
        data, target = data.to(device), target.to(device)

        output = F.log_softmax(loaded_model1(data), dim=1)
        _, predicted = torch.max(output, dim=1)

        true_labels.extend(target.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

/usr/local/lib/python3.10/dist-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


Take Predict to dataframe and Save to csv file

In [ ]:
# สร้าง DataFrame จาก true_labels และ predicted_labels
df_output = pd.DataFrame({'true_label': true_labels, 'predicted_label': predicted_labels})

# บันทึก DataFrame เป็นไฟล์ CSV ในเส้นทางที่กำหนด
output_path = '/content/drive/MyDrive/Csv file/Predict_round1_30.csv'
df_output.to_csv(output_path, index=False)


#Output

In [3]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
df1 = "/content/drive/MyDrive/Csv file/Predict_round1_30.csv"
df2 = "/content/drive/MyDrive/Csv file/clip_start_times.csv"

df1 = pd.read_csv(df1)
df2 = pd.read_csv(df2)

In [ ]:
df_combined = pd.concat([df1, df2], axis=1)
df_combined.drop(columns=['true_label'], inplace=True)
df_combined.index += 1
df_combined

In [ ]:
def get_class_names():
    # สร้างรายชื่อคลาสตามลำดับที่คุณต้องการ
    return ["Buy_Aciton","Class 0 Aciton", "Dead_Aciton", "Defuse Aciton", "Orb Aciton", "Plant Aciton", "Shoot_Aciton", "Skill_Aciton"]

# ใช้ฟังก์ชันเพื่อรับชื่อคลาส
class_names = get_class_names()

# Assuming 'predicted_label' is the name of the column containing predicted labels
# Assuming 'class_names' is a list containing the class names
class_names_dict = {i: class_name for i, class_name in enumerate(class_names)}

# Add a new column 'predicted_class_names' to df_combined
df_combined['predicted_class_names'] = df_combined['predicted_label'].map(class_names_dict)

# เรียกใช้ชื่อคลาสตามตำแหน่งที่ทำนายได้
for label in df_combined['predicted_label']:
    print(class_names[label])
df_combined = df_combined.reindex(columns=['clip_start_times', 'predicted_label', 'predicted_class_names'])

In [22]:
output_path = '/content/drive/MyDrive/Csv file/Final csv for 1 video_round/Predict+time_Round_1.csv'
df_combined.to_csv(output_path, index=False)

In [20]:
df_combined

,clip_start_times,predicted_label,predicted_class_names
1,00:00:00,0,Buy_Aciton
2,00:00:02,0,Buy_Aciton
3,00:00:04,7,Skill_Aciton
4,00:00:06,7,Skill_Aciton
5,00:00:08,7,Skill_Aciton
6,00:00:10,7,Skill_Aciton
7,00:00:12,7,Skill_Aciton
8,00:00:14,7,Skill_Aciton
9,00:00:16,0,Buy_Aciton
10,00:00:18,0,Buy_Aciton
